In [1]:
import sys
import os
import json
import re

import pandas as pd
import numpy as np

In [4]:
os.chdir(r'stocknet-dataset\tweet\raw')

In [3]:
def clean(tweet):
    # tweet = re.sub(r'http\S+|www\S+|@\S+|#\S+|[^A-Za-z\s]', '', tweet)
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)
    tweet=re.sub(r"#(\w+)",'', tweet)
    tweet = re.sub('([_]+)', "", tweet)
    if not re.search(r'[.,!?;:"]$',tweet[-1]):
        tweet+=". "
    return tweet

In [4]:
def extract(ticker,date):
    data=''
    with open(f'{ticker}/{date}') as f:
        for line in f:
            j=json.loads(line)
            if not 'retweeted_status' in j:
                data+=clean(j['text'])
    return data

In [5]:
def extract_ticker_all(ticker):
    data=[]
    for date in os.listdir(ticker):
        data.append((date,extract(ticker,date)))
    return pd.DataFrame(data,columns=['date','tweets'])

In [67]:
for ticker in os.listdir()[2:]:
    data=extract_ticker_all(ticker)
    data.to_csv(f'../clean/{ticker}.csv',index=False)


In [68]:
pd.read_csv('../clean/SLB.csv')

,date,tweets
0,2014-01-02,Not as diversified as $SLB (think $HAL is muc...
1,2014-01-07,$SLB call at $94.50ish was perfect…haven't tri...
2,2014-01-11,S&amp;P100 Performance $BHI $BMY $AEP $SLB $L...
3,2014-01-12,$SLB - China Believes that it Can Duplicate Ou...
4,2014-01-15,Wow. Great on $CCRN $WTI $SLB $VALE Click .
...,...,...
311,2016-03-25,-Cameron deal clears final regulatory hurdle. ...
312,2016-03-27,$SLB:\n\nKey Stocks of the Day: Schlumberger L...
313,2016-03-28,BRIEF-Cameron expresses no opinion on debt ten...
314,2016-03-30,"Limited. Given ""Buy"" Rating at Citigroup Inc...."


In [14]:
os.chdir(r'C:\Users\mingxi.liu\OneDrive - Dymon Asia Capital (Singapore) Pte Ltd\MIDS\W266\stocknet-dataset')

In [4]:
data_all=[]
for i in os.listdir():
    data_all.append(pd.read_csv(i,index_col=0,parse_dates=True)['Adj Close'].rename(i[:-4]))
p=pd.concat(data_all,axis=1)

In [5]:
p.tail()

,AAPL,ABB,ABBV,AEP,AGFS,AMGN,AMZN,BA,BABA,BAC,...,UL,UN,UNH,UPS,UTX,V,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2017-08-28,161.470001,23.07,73.320000,74.099998,7.05,171.779999,946.020020,237.179993,168.300003,23.599287,...,58.119999,59.470001,195.089996,113.790001,115.330002,103.779999,48.610001,51.630001,78.029999,76.470001
2017-08-29,162.910004,22.92,73.889999,73.940002,7.02,172.229996,954.059998,240.490005,167.410004,23.460001,...,58.119999,59.490002,196.729996,114.889999,118.699997,103.769997,48.509998,51.419998,78.769997,76.449997
2017-08-30,163.350006,22.84,73.919998,73.750000,7.03,173.649994,967.590027,240.460007,171.250000,23.870001,...,57.639999,59.020000,195.880005,114.970001,119.599998,103.730003,48.119999,51.360001,78.540001,76.099998
2017-08-31,164.000000,23.18,75.300003,73.629997,7.12,177.770004,980.599976,239.660004,171.740005,23.889999,...,58.209999,59.500000,198.899994,114.360001,119.720001,103.519997,47.970001,51.070000,78.070000,76.330002
2017-09-01,164.050003,23.26,75.419998,73.540001,7.16,178.710007,978.250000,240.330002,171.039993,24.090000,...,58.290001,59.540001,199.750000,114.389999,117.919998,103.900002,47.919998,50.970001,78.370003,76.570000


In [8]:
r_now=np.log(p).diff()['2014':'2015']
r_next=np.log(p).diff(2).shift(-2)['2014':'2015']

In [10]:
def zscore(data):
    return (data-data.mean())/data.std()

In [12]:
r_now=zscore(r_now)
r_next=zscore(r_next)

In [108]:
r=np.log(p).diff(2).shift(-2)['2014':'2015']

In [109]:
upper=r.quantile(0.66)
lower=r.quantile(0.33)


In [110]:
labels=r.copy()

In [111]:
labels[labels>upper]=200
labels[labels<lower]=-200
labels[(labels<=upper)&(labels>=lower)]=0

In [112]:
labels=labels.replace(0,1).replace(-200,0).replace(200,2)

In [113]:
labels.to_csv('price\clean\labels_next2day.csv')

In [97]:
os.chdir('..')

In [98]:
os.getcwd()

'C:\\Users\\mingxi.liu\\OneDrive - Dymon Asia Capital (Singapore) Pte Ltd\\MIDS\\W266\\stocknet-dataset'

In [100]:
pd.read_csv('tweet\clean\AAPL.csv')

,date,tweets
0,2014-01-01,iTV Will Boost Apple $AAPL . users are more ...
1,2014-01-02,$AAPL - Wall St. kicks off New Year on lower n...
2,2014-01-03,The Five-Year-Old iPad: When Will People Stop ...
3,2014-01-04,$AAPL - Boeing contract with machinists - deta...
4,2014-01-05,$AAPL Machinists vote helps Boeing avoid hirin...
...,...,...
779,2016-03-27,Here's how much money Apple makes when you buy...
780,2016-03-28,$VDRM HUGE FILING OUT REVENUES! 1000% run i...
781,2016-03-29,$AAPL Facebook hints at possible Apple Pay tea...
782,2016-03-30,"Top Gainers at Dow: Apple (NASDAQ:AAPL), Micro..."


In [16]:
def match_text_label(ticker,labels_current,labels_next,r_now,r_next):
    try:
        tweets=pd.read_csv(f'tweet/clean/{ticker}.csv',index_col=0,parse_dates=True)
    except FileNotFoundError:
        return
    pairs=pd.concat([tweets,labels_current[ticker].rename('cls_now'),labels_next[ticker].rename('cls_next'),r_now[ticker].rename('reg_now'),r_next[ticker].rename('reg_next')],axis=1)
    pairs['ticker']=ticker
    return pairs.reset_index()

In [15]:
labels_current=pd.read_csv('price/clean/labels_current.csv',parse_dates=True,index_col=0)
labels_next=pd.read_csv('price/clean/labels_next2day.csv',parse_dates=True,index_col=0)

In [18]:
pairs=pd.concat([match_text_label(t,labels_current,labels_next,r_now,r_next) for t in labels_current.columns])

In [22]:
pairs.tail()

,date,tweets,cls_now,cls_next,reg_now,reg_next,ticker
709,2016-03-25,$XOM Analysts Ratings on Exxon Mobil Corporati...,NaN,NaN,NaN,NaN,XOM
710,2016-03-28,Dow Trend $GE $MMM $UNH $TRV $WMT $JNJ $KO $I...,NaN,NaN,NaN,NaN,XOM
711,2016-03-29,$XOM:\n\nExxon Mobil – The Turnaround Has Begu...,NaN,NaN,NaN,NaN,XOM
712,2016-03-30,NEW POST: Proving Oil's Guilt $DWTI $SPY $USO...,NaN,NaN,NaN,NaN,XOM
713,2016-03-31,$XOM Shares in Focus: Exxon Mobil Corporation ...,NaN,NaN,NaN,NaN,XOM


In [21]:
pairs.columns=['date', 'tweets', 'cls_now', 'cls_next', 'reg_now', 'reg_next',
       'ticker']

In [24]:
pairs.dropna().to_csv('clean_data.csv',index=False)

In [136]:
pd.read_csv('clean_data.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     18359 non-null  object 
 1   tweets   18359 non-null  object 
 2   current  18359 non-null  float64
 3   next     18359 non-null  float64
 4   ticker   18359 non-null  object 
dtypes: float64(2), object(3)
memory usage: 717.3+ KB
